# Qwen2模型验证

In [1]:
from modelscope import AutoModelForCausalLM,AutoTokenizer
import torch
import os
os.environ['MODELSCOPE_CACHE'] = r'D:\myProject\modelscope_hub'
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "qwen/Qwen2-0.5B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("qwen/Qwen2-0.5B-Instruct")



2026-02-25 14:23:35,912 - modelscope - INFO - Creating symbolic link [D:\myProject\modelscope_hub\models\qwen\Qwen2-0.5B-Instruct].
2026-02-25 14:23:35,913 - modelscope - WARNING - Failed to create symbolic link D:\myProject\modelscope_hub\models\qwen\Qwen2-0.5B-Instruct for D:\myProject\modelscope_hub\models\qwen\Qwen2-0___5B-Instruct.
`torch_dtype` is deprecated! Use `dtype` instead!


2026-02-25 14:23:39,939 - modelscope - INFO - Creating symbolic link [D:\myProject\modelscope_hub\models\qwen\Qwen2-0.5B-Instruct].
2026-02-25 14:23:39,941 - modelscope - WARNING - Failed to create symbolic link D:\myProject\modelscope_hub\models\qwen\Qwen2-0.5B-Instruct for D:\myProject\modelscope_hub\models\qwen\Qwen2-0___5B-Instruct.


In [2]:
print(tokenizer)

Qwen2TokenizerFast(name_or_path='D:\myProject\modelscope_hub\models\qwen\Qwen2-0___5B-Instruct', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)


In [5]:
def chat(prompt):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    # print(text)
    # print('-'*30)
    with torch.no_grad():
        model_inputs = tokenizer([text], return_tensors="pt").to(device)
        # print(model_inputs)
        # print('-'*30)

        
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512
        )
        
        # print('total',generated_ids)
        # print('-'*30)
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        # print('generated',generated_ids)
        # print('-'*30)

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        torch.cuda.empty_cache()
        return response

In [6]:
prompt = "你是谁？"
chat(prompt)

'我是来自阿里云的大规模语言模型，我叫通义千问。'

In [7]:
print(chat('2+2等于几'))
print(chat('22+22等于几'))
print(chat('3+3等于几'))


2 + 2 等于 4。
22 + 22 equals 44。
3 + 3 等于 6。


In [ ]:
def batch_chat(prompts):
    """
    批量推理实现，不用for循环。
    输入: prompts: List[str]
    输出: responses: List[str]
    """
    batch_messages = [
        [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": p},
        ]
        for p in prompts
    ]
    texts = tokenizer.apply_chat_template(
        batch_messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    
    # texts: List[str]，适合直接 batch
    with torch.no_grad():
        model_inputs = tokenizer(texts, return_tensors="pt", padding=True).to(device)
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512
        )
        # 对于批量，截取每个结果的 generated 部分
        input_lens = [len(ids) for ids in model_inputs.input_ids]
        gen_ids = []
        for idx, output_ids in enumerate(generated_ids):
            gen_ids.append(output_ids[input_lens[idx]:])
        responses = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
        return responses

# 示例:
batch_prompts = ["2+2等于几", "22+22等于几", "3+3等于几"]
results = batch_chat(batch_prompts)
for p, r in zip(batch_prompts, results):
    print(f"Q: {p}\nA: {r}\n")

In [17]:
batch_prompts = ["2+2等于几", "22+22等于几", "3+3等于几",'你是什么模型']
batch_messages = [
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": p},
    ]
    for p in batch_prompts
]
texts = tokenizer.apply_chat_template(
    batch_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer(texts, return_tensors="pt",padding=True,padding_side='left').to(device)
(tokenizer.batch_decode(model_inputs['input_ids']))

['<|endoftext|><|endoftext|><|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n2+2等于几<|im_end|>\n<|im_start|>assistant\n',
 '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n22+22等于几<|im_end|>\n<|im_start|>assistant\n',
 '<|endoftext|><|endoftext|><|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n3+3等于几<|im_end|>\n<|im_start|>assistant\n',
 '<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n你是什么模型<|im_end|>\n<|im_start|>assistant\n']

In [18]:
with torch.no_grad():
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )

In [19]:
tokenizer.batch_decode(generated_ids,skip_special_tokens=True)

['system\nYou are a helpful assistant.\nuser\n2+2等于几\nassistant\n',
 'system\nYou are a helpful assistant.\nuser\n22+22等于几\nassistant\n33333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333',
 'system\nYou are a helpful assistant.\nuser\n3+3等于几\nassistant\n3+3等于3333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333

In [20]:
input_lens = [len(ids) for ids in model_inputs.input_ids]
gen_ids = []
for idx, output_ids in enumerate(generated_ids):
    gen_ids.append(output_ids[input_lens[idx]:])
responses = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
responses

['',
 '33333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333',
 '3+3等于333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333

In [9]:
# 训练数据预处理方法
def preprocess(tokenizer,batch_messages):
    input_list=[]
    target_list=[]
    
    im_start=tokenizer('<|im_start|>').input_ids
    im_end=tokenizer('<|im_end|>').input_ids
    newline=tokenizer('\n').input_ids
    pad=tokenizer('<|endoftext|>').input_ids
    ignore=[-100]
    
    for group in batch_messages:
        input_ids=[]
        target_ids=[]
        for msg in group:
            role=tokenizer(msg['role']).input_ids
            content=tokenizer(msg['content']).input_ids
            if msg['role'] in ['system','user']:
                ignore_parts=role+newline+content
                input_ids+=im_start+ignore_parts+im_end+newline
                target_ids+=im_start+ignore*len(ignore_parts)+im_end+newline
            else:
                ignore_parts=role+newline
                input_ids+=im_start+ignore_parts+content+im_end+newline
                target_ids+=im_start+ignore*len(ignore_parts)+content+im_end+newline
        input_list.append(input_ids)
        target_list.append(target_ids)
    
    # padding
    max_len=max([len(ids) for ids in input_list])
    for input_ids,target_ids in zip(input_list,target_list):
        input_ids+=pad*(max_len-len(input_ids))
        target_ids+=ignore*(max_len-len(target_ids))
    batch_input_ids=torch.tensor(input_list,dtype=torch.long)
    batch_target_ids=torch.tensor(target_list,dtype=torch.long)
    batch_mask=batch_input_ids.ne(pad[0]).type(torch.long)
    return batch_input_ids,batch_target_ids,batch_mask

In [10]:
prompt = "2+2等于几"
messages = [
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": '2+2等于5。'},
    ],
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": '3+3等于7。'},
    ]
]

model.eval()

batch_input_ids,batch_target_ids,batch_mask=preprocess(tokenizer,messages)


# 训练数据测试

In [12]:

batch_input_ids,batch_target_ids,batch_mask=preprocess(tokenizer,messages)
model_outputs=model(batch_input_ids.to(device))

output_tokens=model_outputs.logits.argmax(dim=-1)

logits=model_outputs.logits[:,:-1,:]
targets=batch_target_ids[:,1:].to(device)
print('logits:',logits.shape) # 模型输出
print('targets:',targets.shape) # 拟合目标


logits: torch.Size([2, 31, 151936])
targets: torch.Size([2, 31])


In [13]:

from torch.nn import CrossEntropyLoss

# 损失
loss_fn=CrossEntropyLoss()
loss=loss_fn(logits.reshape(-1,logits.size(2)),targets.reshape(-1))
print('loss:',loss)

# 优化器
optimizer=torch.optim.SGD(model.parameters())
optimizer.zero_grad()

# 求梯度
loss.backward()

# 梯度下降
optimizer.step()

loss: tensor(2.2188, device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<NllLossBackward0>)


In [14]:
chat('2+2等于')

'33333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333'

In [15]:
chat('2+3等于')


'3+3等于3333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333'